<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%2012/create_LQ_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Lapham's Quarterly Corpus

In [0]:
# !pip3 install PyPDF2

In [0]:
# import re, string, glob
# import PyPDF2
# import nltk
# nltk.download('punkt')
# from nltk import sent_tokenize

In [0]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/Drive', force_remount = True)
# # When you run this block, you will need to click open a link to get some 

In [0]:
# !ls 'Drive/My Drive/CIS-530/Homework 12/'

In [0]:
# import os
# os.chdir('Drive/My Drive/CIS-530/Homework 12/')

In [0]:
# !ls 'Data/LQIssues/'

In [0]:
# LQ_dump_path = 'Data/LQIssues/*'
# IDs = glob.glob(LQ_dump_path)
# corpus = []
# for ID in IDs:
#     print(ID)
#     file_path = ID
#     pdf_file_obj = open(file_path,'rb')
#     pdf_reader = PyPDF2.PdfFileReader(pdf_file_obj)
#     if pdf_reader.isEncrypted:
#         pdf_reader.decrypt('')
#     num_pages = pdf_reader.numPages    
#     pdf_text = []
#     for i in range(num_pages):
#         page_obj = pdf_reader.getPage(i)
#         pdf_text.append(page_obj.extractText())
#     pdf_file_obj.close()
#     del pdf_text[0:9] # generally removes pages up to "About the Contributors"
#     len_text = len(pdf_text)
#     for i in range(len_text):
#         pdf_text[i] = pdf_text[i].replace("\n-\n","")
#         pdf_text[i] = pdf_text[i].replace("\n-","")
#         sent = sent_tokenize(pdf_text[i])
#         corpus.append(sent)

In [0]:
# corpus_path = 'Data/LQIssues/LQCorpus.txt'
# fid = open(corpus_path,"w+")
# for i in range(len(corpus)):
#     for j in range(len(corpus[i])):
#         element = corpus[i][j]
#         fid.write(element)
#         fid.write(" ")
# fid.close()

In [0]:
# from itertools import islice

# corpus_path = 'Data/LQIssues/LQCorpus.txt'
# file = open(corpus_path, encoding = 'UTF-8', errors = 'ignore').readlines()
# file_len = len(file)
# num_train_lines = round(0.90 * file_len)
# num_valid_lines = round(0.05 * file_len)
# num_test_lines = file_len - num_train_lines - num_valid_lines
# lines_in_sets = [num_train_lines, num_valid_lines, num_test_lines]
# temp = iter(file) 
# splits = [list(islice(temp, 0, ele)) for ele in lines_in_sets] 
# # split 0 is training data
# with open('Data/LQIssues/laphams_quarterly_train.txt', 'w') as f:
#   for line in splits[0]:
#     f.write('%s\n' % line)
# # split 1 is validation data
# with open('Data/LQIssues/laphams_quarterly_valid.txt', 'w') as f:
#   for line in splits[1]:
#     f.write('%s\n' % line)
# # split 2 is test data
# with open('Data/LQIssues/laphams_quarterly_test.txt', 'w') as f:
#   for line in splits[2]:
#     f.write('%s\n' % line)

# Fine-Tune and Evaluate LQ Corpus Model

In [1]:
!git clone https://github.com/huggingface/transformers

import os
os.chdir('/content/transformers')

!git checkout d6ef587a10e0d8836376a2314d8aeae36ad63263

!pip install .
!pip install -r ./examples/requirements.txt

os.chdir('/content/transformers/examples')

!pip install dict_to_obj

Cloning into 'transformers'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 25199 (delta 42), reused 50 (delta 26), pack-reused 25119
Receiving objects: 100% (25199/25199), 15.17 MiB | 9.06 MiB/s, done.
Resolving deltas: 100% (17675/17675), done.
Note: checking out 'd6ef587a10e0d8836376a2314d8aeae36ad63263'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at d6ef587a [ci] Fixup e36bd94345af6045108a391f9ac7f4dc557548de
Processing /content/transformers
     |████████████████████████████████| 3.7MB 2.8MB/s 
     |█████████████

In [0]:
import torch
import run_language_modeling
import run_generation
from dict_to_obj import DictToObj
import collections
import random
import numpy as np

from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls "/content/drive/My Drive/finetuned_models/laphams_quarterly"

checkpoint-3600  checkpoint-4000   pytorch_model.bin	    vocab.json
checkpoint-3700  config.json	   special_tokens_map.json
checkpoint-3800  eval_results.txt  tokenizer_config.json
checkpoint-3900  merges.txt	   training_args.bin


In [5]:
!ls '/content/drive/My Drive/CIS-530/Homework 12/Data'

LQIssues		 text_adventures_test.txt
text_adventures_dev.txt  text_adventures_train.txt


In [0]:
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/finetuned_models/laphams_quarterly' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --logging_steps=500 \
    --save_steps=100 \
    --train_data_file=/content/drive/My\ Drive/CIS-530/Homework\ 12/Data/LQIssues/laphams_quarterly_train.txt \
    --do_eval \
    --eval_data_file=/content/drive/My\ Drive/CIS-530/Homework\ 12/Data/LQIssues/laphams_quarterly_valid.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=128 \
    --gradient_accumulation_steps=5 \
    --overwrite_output_dir

Streaming output truncated to the last 5000 lines.
Iteration:  76% 15323/20139 [36:18<09:25,  8.52it/s]
Iteration:  76% 15324/20139 [36:18<09:08,  8.77it/s]
Iteration:  76% 15325/20139 [36:18<10:03,  7.98it/s]
Iteration:  76% 15326/20139 [36:18<10:27,  7.67it/s]
Iteration:  76% 15327/20139 [36:19<09:50,  8.16it/s]
Iteration:  76% 15328/20139 [36:19<09:24,  8.52it/s]
Iteration:  76% 15329/20139 [36:19<09:10,  8.74it/s]
Iteration:  76% 15330/20139 [36:19<10:07,  7.92it/s]
Iteration:  76% 15331/20139 [36:19<10:27,  7.66it/s]
Iteration:  76% 15332/20139 [36:19<09:53,  8.10it/s]
Iteration:  76% 15333/20139 [36:19<09:29,  8.44it/s]
Iteration:  76% 15334/20139 [36:19<09:08,  8.76it/s]
Iteration:  76% 15335/20139 [36:20<09:59,  8.02it/s]
Iteration:  76% 15336/20139 [36:20<10:22,  7.71it/s]
Iteration:  76% 15337/20139 [36:20<09:49,  8.14it/s]
Iteration:  76% 15338/20139 [36:20<09:25,  8.49it/s]
Iteration:  76% 15339/20139 [36:20<09:07,  8.76it/s]
Iteration:  76% 15340/20139 [36:20<10:00,  7.99i

In [6]:
!ls '/content/drive/My Drive/finetuned_models/laphams_quarterly'

checkpoint-3600  checkpoint-4000   pytorch_model.bin	    vocab.json
checkpoint-3700  config.json	   special_tokens_map.json
checkpoint-3800  eval_results.txt  tokenizer_config.json
checkpoint-3900  merges.txt	   training_args.bin


In [0]:
def load_model(args):
  """Creates a model and loads in weights for it."""
  config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=None)

  model = AutoModelWithLMHead.from_pretrained(
      args.model_name_or_path,
      from_tf=bool(".ckpt" in args.model_name_or_path),
      config=config,
      cache_dir=None
  )

  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)
  config = AutoConfig.from_pretrained(args.model_name_or_path, cache_dir=None)
  model = AutoModelWithLMHead.from_pretrained(
    args.model_name_or_path,from_tf=bool(".ckpt" in args.model_name_or_path),
    config=config, cache_dir=None)
  
  model.to(args.device)
  return model

def set_seed(seed):
  """Set the random seed."""
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if args.n_gpu > 0:
    torch.cuda.manual_seed_all(args.seed)

def do_perplexity_eval(args, model, data_file_path):
  """Computes the perplexity of the text in data_file_path according to the provided model."""
  set_seed(args.seed)

  args.eval_data_file=data_file_path

  tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, cache_dir=None)

  args.block_size = min(args.block_size, tokenizer.max_len)

  result = run_language_modeling.evaluate(args, model, tokenizer, prefix="")
  return result

In [0]:
# Set this to the checkpoint you want to evalute, or to "gpt2-medium" to
# evaluate the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/laphams_quarterly/checkpoint-4000'
# CHECKPOINT_PATH = "gpt2"

# Set this to the list of text files you want to evaluate the perplexity of.
DATA_PATHS = ["/content/drive/My Drive/CIS-530/Homework 12/Data/LQIssues/laphams_quarterly_valid.txt",
              "/content/drive/My Drive/CIS-530/Homework 12/Data/LQIssues/laphams_quarterly_test.txt"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  block_size = 128,
  local_rank=-1,
  eval_batch_size=2,
  per_gpu_eval_batch_size=2,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  line_by_line=False,
  overwrite_cache=None,
  model_type='gpt2',
  seed=42,
)
args = DictToObj(args)

model = load_model(args)

for data_path in DATA_PATHS:
  eval_results = do_perplexity_eval(args, model, data_path)
  perplexity = eval_results['perplexity']
  print('{} is the perplexity of {} according to {}'.format(
      perplexity, data_path, CHECKPOINT_PATH))

04/28/2020 23:16:00 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/laphams_quarterly/checkpoint-4000/config.json
04/28/2020 23:16:00 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": false,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "eos_token_ids": null,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_beams": 1,
  "num_labels": 2,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states

Running on device:  cuda


04/28/2020 23:16:20 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/laphams_quarterly/checkpoint-4000/config.json
04/28/2020 23:16:20 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": false,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "eos_token_ids": null,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_beams": 1,
  "num_labels": 2,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states

20.183937072753906 is the perplexity of /content/drive/My Drive/CIS-530/Homework 12/Data/LQIssues/laphams_quarterly_valid.txt according to /content/drive/My Drive/finetuned_models/laphams_quarterly/checkpoint-4000



Evaluating: 100%|██████████| 1107/1107 [00:33<00:00, 32.84it/s]
04/28/2020 23:17:46 - INFO - run_language_modeling -   ***** Eval results  *****
04/28/2020 23:17:46 - INFO - run_language_modeling -     perplexity = tensor(21.2911)


21.291067123413086 is the perplexity of /content/drive/My Drive/CIS-530/Homework 12/Data/LQIssues/laphams_quarterly_test.txt according to /content/drive/My Drive/finetuned_models/laphams_quarterly/checkpoint-4000


In [0]:
def generate_samples(args, model, prompt_text):
  """Generating sampling for the provided prompt using the provided model."""
  set_seed(args.seed)

  _, tokenizer_class = run_generation.MODEL_CLASSES[args.model_type]
  tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  requires_preprocessing = args.model_type in run_generation.PREPROCESSING_FUNCTIONS.keys()
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(args.device)

  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=args.length + len(encoded_prompt[0]),
      temperature=args.temperature,
      top_k=args.k,
      top_p=args.p,
      repetition_penalty=args.repetition_penalty,
      do_sample=True,
      num_return_sequences=args.num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(args.stop_token) if args.stop_token else None]

    # Remove the excess text that was used for pre-processing
    text = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]

    # Add the prompt at the beginning of the sequence.
    total_sequence = prompt_text + text

    generated_sequences.append(total_sequence)

  return generated_sequences

In [12]:
# Set this to the checkpoint you want to use for generation, or to "gpt2-medium"
# to generate with the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/laphams_quarterly/checkpoint-4000'
# CHECKPOINT_PATH = 'gpt2'

# You should try out other prompts as well as no prompt at all.
PROMPT = 'Machiavelli during his lifetime'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  model_type='gpt2',
  seed=42,
  stop_token=None, # Set this if your dataset has a special word that indicates the end of a text.
  temperature=1.0,  # temperature sampling. Set this to temperature=1.0 to not use temperature.
  k=50,  # k for top-k sampling. Set this to k=0 to not use top-k.
  p=1.0,  # p for nucleus sampling. Set this to p=1.0 to not use nucleus sampling.
  repetition_penalty=None,
  length=100,  # Number of tokens to generate.
  num_return_sequences=3,  # Number of independently computed samples to generate.
)
args = DictToObj(args)

model = load_model(args)
sequences = generate_samples(args, model, PROMPT)
for idx, sequence in enumerate(sequences):
  print('\n====== GENERATION {} ======'.format(idx))
  print(sequence)

04/30/2020 06:45:37 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/laphams_quarterly/checkpoint-4000/config.json
04/30/2020 06:45:37 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": false,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "eos_token_ids": null,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_beams": 1,
  "num_labels": 2,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states

Running on device:  cuda


04/30/2020 06:45:43 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/laphams_quarterly/checkpoint-4000/config.json
04/30/2020 06:45:43 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": false,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "eos_token_ids": null,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_beams": 1,
  "num_labels": 2,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states


====== GENERATION 0 ======
Machiavelli during his lifetime. ﬁ

I don't know what the 

great age of music is,ﬂ he wrote, ﬁbut I know that it is an 

essentia of music.ﬂ ˜ere are 

many things that the age of music has gone backward 

to: ﬁ˛ee, with the sound of a woman,ﬂ to ﬁ˛ee, 

with

====== GENERATION 1 ======
Machiavelli during his lifetime, 

wrote 

that 

the ˛fth novel of his career was 

˜e 



T

reatise of Man

 ˚tt the 

S

hakespearean 

S

helf

, composed on the same days as the 

G

anzo-

tian war, was 

 
˜m 

˜n

====== GENERATION 2 ======
Machiavelli during his lifetime. An icon, 



he was the world™s most 

revered explorer, a pioneer of the 

mauvaise, a pioneer of the spirit, an explorer of the 



world™s beauty. ˚en 



his work of exploration, his vast 



laboratory of the Holy GrailŠ



the GrailŠhas created the 

world™s largest collection of
